In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from txtox.utils import get_paths
import scanpy as sc
from sklearn.neighbors import KNeighborsTransformer


path = get_paths()["data_root"] + "VISp.h5ad"
adata = ad.read_h5ad(path)
adata.obsm["spatial"] = adata.obs[["x_section", "y_section", "z_section"]].values

/home/rohan/miniconda3/envs/txtox-torch-gpu/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [2]:
sc.pp.neighbors(
    adata,
    n_neighbors=15,
    n_pcs=None,
    use_rep="spatial",
    knn=True,
    transformer=KNeighborsTransformer(n_neighbors=15, metric="minkowski", p=2),
    key_added="spatial",
    copy=False,
)

In [3]:
adata.write_h5ad(get_paths()["data_root"] + "VISp_nhood.h5ad")

In [4]:
nhood_idx = adata.obsp["spatial_connectivities"][0, :].toarray()
nhood_idx = np.where(nhood_idx)[1]
dists = np.linalg.norm(adata.obsm["spatial"][nhood_idx] - adata.obsm["spatial"][0], axis=1) * 1e3

nhood_dist = adata.obsp["spatial_distances"][0, :].toarray()
nhood_dist = nhood_dist[:, nhood_idx]
dists_ = nhood_dist * 1e3

assert np.allclose(dists, dists_, rtol=1e-5, atol=1e-8), "Distances do not match"